**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Jun  4 13:15:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'2023 SW중심대학 공동AI경진대회 지원서(Crypto 팀).docx'   yolov3
'Colab Notebooks'					 '족보 및 솔루션'
 Screenshot_20230412-175741.jpg


**1) Clone the Darknet**



In [4]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15521, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 15521 (delta 0), reused 4 (delta 0), pack-reused 15514
Receiving objects: 100% (15521/15521), 14.17 MiB | 23.41 MiB/s, done.
Resolving deltas: 100% (10413/10413), done.


# 새 섹션

**2) Compile Darknet using Nvidia GPU**


In [6]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |           

**3) Configure Darknet network for training YOLO V3**

In [40]:
#!cp cfg/yolov3.cfg cfg/yolov3_training.cfg
!cp /mydrive/yolov3/data/yolov3_testing.cfg cfg/yolov3_training.cfg

In [41]:
!echo -e '' > data/obj.names
!echo -e '' > data/obj.data
!cp /mydrive/yolov3/data/obj.names data/obj.names
!cp /mydrive/yolov3/data/obj.data data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [42]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-06-04 13:43:38--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.4’

darknet53.conv.74.4 100%[===================>] 154.96M  90.4MB/s    in 1.7s    

2023-06-04 13:43:40 (90.4 MB/s) - ‘darknet53.conv.74.4’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [48]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
 extracting: data/obj/classes.txt    
  inflating: data/obj/drone_1685880477.jpg  
 extracting: data/obj/drone_1685880477.txt  
  inflating: data/obj/drone_1685880479.jpg  
  inflating: data/obj/drone_1685880479.txt  
  inflating: data/obj/drone_1685880481.jpg  
 extracting: data/obj/drone_1685880481.txt  
  inflating: data/obj/drone_1685880483.jpg  
  inflating: data/obj/drone_1685880483.txt  
  inflating: data/obj/drone_1685880485.jpg  
 extracting: data/obj/drone_1685880485.txt  
  inflating: data/obj/drone_1685880489.jpg  
  inflating: data/obj/drone_1685880489.txt  
  inflating: data/obj/drone_1685880491.jpg  
  inflating: data/obj/drone_1685880491.txt  
  inflating: data/obj/drone_1685880493.jpg  
 extracting: data/obj/drone_1685880493.txt  
  inflating: data/obj/drone_1685880495.jpg  
 extracting: data/obj/drone_1685880495.txt  
  inflating: data/obj/drone_1685880497.jpg  
  inflating: data/obj/drone_1685880497.txt  
  inflating: data/obj/dro

In [49]:
import glob
images_list = glob.glob("data/obj/*.jpg")
#images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/drone_1685880554.jpg', 'data/obj/drone_1685880877.jpg', 'data/obj/drone_1685880841.jpg', 'data/obj/drone_1685880657.jpg', 'data/obj/drone_1685880881.jpg', 'data/obj/drone_1685880538.jpg', 'data/obj/drone_1685880823.jpg', 'data/obj/drone_1685880779.jpg', 'data/obj/drone_1685880479.jpg', 'data/obj/drone_1685880518.jpg', 'data/obj/drone_1685880532.jpg', 'data/obj/drone_1685880648.jpg', 'data/obj/drone_1685880831.jpg', 'data/obj/drone_1685880572.jpg', 'data/obj/drone_1685880756.jpg', 'data/obj/drone_1685880497.jpg', 'data/obj/drone_1685880586.jpg', 'data/obj/drone_1685880606.jpg', 'data/obj/drone_1685880665.jpg', 'data/obj/drone_1685880608.jpg', 'data/obj/drone_1685880564.jpg', 'data/obj/drone_1685880917.jpg', 'data/obj/drone_1685880887.jpg', 'data/obj/drone_1685880787.jpg', 'data/obj/drone_1685880905.jpg', 'data/obj/drone_1685880762.jpg', 'data/obj/drone_1685880913.jpg', 'data/obj/drone_1685880777.jpg', 'data/obj/drone_1685880861.jpg', 'data/obj/drone_1685880813.jpg', 'data/obj

In [50]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close()

**6) Start the training**

In [51]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.849902), count: 5, class_loss = 0.158478, iou_loss = 0.055409, total_loss = 0.213887 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.870802), count: 2, class_loss = 0.004541, iou_loss = 0.015032, total_loss = 0.019574 
 total_bbox = 25258, rewritten_bbox = 0.000000 % 
]2;3335/4000: loss=0.1 hours left=0.1
 3335: 0.078283, 0.105773 avg loss, 0.000100 rate, 0.396299 seconds, 26680 images, 0.114205 hours left
Loaded: 0.000084 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.861771), count: 1, class_loss = 0.005326, iou_loss = 0.006514, total_loss = 0.011840 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.811480), count: 4, class_loss = 0.137847, iou_loss = 0.065156, total_loss = 0.203003 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Av